In [1]:
import sys
sys.path.append(r'C:\Users\cosmosyw\Documents\Softwares')

import os
import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd
import numpy as np
import copy
import time

import matplotlib.pyplot as plt

19560


# 1. Load Data

## Load the spot df folder

In [2]:
para_subfolder =r'CW_v3'

main_folder = r'E:\DNA_analysis'
exp_folder_names = ['20220729_Mecp2_0724_DNA', '20220819_Mecp2_DNA_from_0809', '20230119_Ma_1027_DNA', '20230316_Mb_1103_DNA']
exp_folders = [os.path.join(main_folder, _fl) for _fl in exp_folder_names]
picked_spot_folders = []
for fd in exp_folders:
    for _analysis_fd in os.listdir(fd):
        if 'Analysis_' in _analysis_fd:
            picked_spot_folders.append(os.path.join(fd, _analysis_fd))
picked_spot_folders = [os.path.join(_fl,'Picked', para_subfolder) for _fl in picked_spot_folders]
picked_spot_folders

['E:\\DNA_analysis\\20220729_Mecp2_0724_DNA\\Analysis_0507\\Picked\\CW_v3',
 'E:\\DNA_analysis\\20220819_Mecp2_DNA_from_0809\\Analysis_0508\\Picked\\CW_v3',
 'E:\\DNA_analysis\\20230119_Ma_1027_DNA\\Analysis_0505\\Picked\\CW_v3',
 'E:\\DNA_analysis\\20230316_Mb_1103_DNA\\Analysis_0506\\Picked\\CW_v3']

# 2. Import functions

In [3]:
import jie
from jie.jie import aligner

from jie.jie import plot

In [4]:
import multiprocessing as mp
import tqdm

num_threads = 16

# 3. Sort the picked spots for each exp replicate

## Exp 0729

### load data

In [5]:
dna_exp_key= '20220729'

# output dict for filename, file
output_df_dict = {}

exp_picked_spot_folder = [_f for _f in picked_spot_folders if dna_exp_key in _f][0]

if os.path.exists(exp_picked_spot_folder):
    file_list = [os.path.join(exp_picked_spot_folder, _fl) for _fl in os.listdir(exp_picked_spot_folder) if '.h5' in _fl]
    for file in file_list:
        output_df_dict[os.path.basename(file)]=pd.read_hdf(file)
        
exp_picked_spot_folder

'E:\\DNA_analysis\\20220729_Mecp2_0724_DNA\\Analysis_0507\\Picked\\CW_v3'

In [6]:
output_df_dict.keys()

dict_keys(['20220729exp_pick_chr1.h5', '20220729exp_pick_chr10.h5', '20220729exp_pick_chr11.h5', '20220729exp_pick_chr12.h5', '20220729exp_pick_chr13.h5', '20220729exp_pick_chr14.h5', '20220729exp_pick_chr15.h5', '20220729exp_pick_chr16.h5', '20220729exp_pick_chr17.h5', '20220729exp_pick_chr18.h5', '20220729exp_pick_chr19.h5', '20220729exp_pick_chr2.h5', '20220729exp_pick_chr3.h5', '20220729exp_pick_chr4.h5', '20220729exp_pick_chr5.h5', '20220729exp_pick_chr6.h5', '20220729exp_pick_chr7.h5', '20220729exp_pick_chr8.h5', '20220729exp_pick_chr9.h5', '20220729exp_pick_chrX.h5'])

In [7]:
output_df_dict[list(output_df_dict.keys())[0]]

,replicate,FOV,orig_cellID,finalcellID,celltype,chr,hyb,region_name,x_hat,y_hat,z_hat,sig_x,sig_y,sig_z,rowID,MeCP2_genotype,fiberidx
0,20220729,160,Cell-19,100072942413897134904184107996429957963,Astro,chr1,0,1:3742742-3759944,1545.767100,759.435970,28.892279,1,1,1,16755862,WT,1
8,20220729,160,Cell-19,100072942413897134904184107996429957963,Astro,chr1,3,1:9627926-9637875,1551.079633,760.952200,22.569937,1,1,1,16758307,WT,1
25,20220729,160,Cell-19,100072942413897134904184107996429957963,Astro,chr1,9,1:20920666-20929358,1563.958600,762.849700,25.900356,1,1,1,16759627,WT,1
34,20220729,160,Cell-19,100072942413897134904184107996429957963,Astro,chr1,11,1:21522568-21534512,1570.275800,762.627197,21.983661,1,1,1,16757879,WT,1
37,20220729,160,Cell-19,100072942413897134904184107996429957963,Astro,chr1,12,1:21732182-21745770,1563.924833,762.916710,25.909013,1,1,1,16757458,WT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,20220729,59,Cell-54,94240242758590968427651986201582734953,Vip,chr1,145,1:188751447-188759965,1011.516920,1803.282200,25.717475,1,1,1,27916412,WT,2
521,20220729,59,Cell-54,94240242758590968427651986201582734953,Vip,chr1,146,1:189040928-189050246,1019.154845,1810.248500,25.141552,1,1,1,27921899,WT,2
533,20220729,59,Cell-54,94240242758590968427651986201582734953,Vip,chr1,148,1:189510684-189521211,1026.831900,1807.450800,31.602654,1,1,1,27921191,WT,2
540,20220729,59,Cell-54,94240242758590968427651986201582734953,Vip,chr1,150,1:191250765-191259922,1025.402500,1812.484500,28.427424,1,1,1,27917064,WT,2


### process data

In [8]:
# define output folder
output_folder = exp_picked_spot_folder.replace(para_subfolder, f'{para_subfolder}_sorted')
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    print ('Generating output folders')
    
# loop through all chr h5-df    
for fname, chr_df_all in output_df_dict.items():
    
    output_fname = os.path.join(output_folder, fname)
    chosen_chrom = chr_df_all['chr'].values[0]
    
    chr_df_cell_list = []
    # group by celltype, finalcellD
    for i, (name, group) in tqdm.tqdm(enumerate(chr_df_all.groupby(by=['celltype','finalcellID']))):
        chr_df_cell = copy.deepcopy(group)
        chr_df_cell_list.append(chr_df_cell)
        
    del chr_df_all
    print(f'Start multiprocessing chromosome {chosen_chrom}.')
    start_time = time.time()
    with mp.Pool(num_threads) as sort_pool:
        _res = sort_pool.starmap(aligner.sort_spots_by_fiber_density, zip(chr_df_cell_list), chunksize=1)
        sort_pool.close()
        sort_pool.join()
        sort_pool.terminate()

    elapsed_time = time.time() - start_time
    print(f'Finish processing chromosome {chosen_chrom}.')
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    sorted_chr_df_all = pd.concat(_res)
    sorted_chr_df_all.to_hdf(output_fname, key='df', mode='w')
    #del(sorted_chr_df_all)
    print(f'Finish saving chromosome {chosen_chrom}.')
    print(f'+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    


Generating output folders


4805it [00:00, 12743.44it/s]


Start multiprocessing chromosome chr1.
Finish processing chromosome chr1.
00:02:36.38
Finish saving chromosome chr1.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


5007it [00:00, 10954.83it/s]


Start multiprocessing chromosome chr10.
Finish processing chromosome chr10.
00:01:43.16
Finish saving chromosome chr10.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


...

Start multiprocessing chromosome chr8.
Finish processing chromosome chr8.
00:01:11.67
Finish saving chromosome chr8.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


4530it [00:00, 11705.45it/s]


Start multiprocessing chromosome chr9.
Finish processing chromosome chr9.
00:01:03.22
Finish saving chromosome chr9.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


4560it [00:00, 21408.49it/s]


Start multiprocessing chromosome chrX.
Finish processing chromosome chrX.
00:00:31.89
Finish saving chromosome chrX.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


## Exp 0819

### load data

In [9]:
dna_exp_key= '20220819'

# output dict for filename, file
output_df_dict = {}

exp_picked_spot_folder = [_f for _f in picked_spot_folders if dna_exp_key in _f][0]

if os.path.exists(exp_picked_spot_folder):
    file_list = [os.path.join(exp_picked_spot_folder, _fl) for _fl in os.listdir(exp_picked_spot_folder) if '.h5' in _fl]
    for file in file_list:
        output_df_dict[os.path.basename(file)]=pd.read_hdf(file)
        
exp_picked_spot_folder

'E:\\DNA_analysis\\20220819_Mecp2_DNA_from_0809\\Analysis_0508\\Picked\\CW_v3'

### process data

In [10]:
# define output folder
output_folder = exp_picked_spot_folder.replace(para_subfolder, f'{para_subfolder}_sorted')
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    print ('Generating output folders')
    
# loop through all chr h5-df    
for fname, chr_df_all in output_df_dict.items():
    
    output_fname = os.path.join(output_folder, fname)
    chosen_chrom = chr_df_all['chr'].values[0]
    
    chr_df_cell_list = []
    # group by celltype, finalcellD
    for i, (name, group) in tqdm.tqdm(enumerate(chr_df_all.groupby(by=['celltype','finalcellID']))):
        chr_df_cell = copy.deepcopy(group)
        chr_df_cell_list.append(chr_df_cell)
        
    del chr_df_all
    print(f'Start multiprocessing chromosome {chosen_chrom}.')
    start_time = time.time()
    with mp.Pool(num_threads) as sort_pool:
        _res = sort_pool.starmap(aligner.sort_spots_by_fiber_density, zip(chr_df_cell_list), chunksize=1)
        sort_pool.close()
        sort_pool.join()
        sort_pool.terminate()

    elapsed_time = time.time() - start_time
    print(f'Finish processing chromosome {chosen_chrom}.')
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    sorted_chr_df_all = pd.concat(_res)
    sorted_chr_df_all.to_hdf(output_fname, key='df', mode='w')
    #del(sorted_chr_df_all)
    print(f'Finish saving chromosome {chosen_chrom}.')
    print(f'+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    


Generating output folders


9176it [00:00, 12316.79it/s]


Start multiprocessing chromosome chr1.
Finish processing chromosome chr1.
00:03:59.30
Finish saving chromosome chr1.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


9831it [00:00, 15629.57it/s]


Start multiprocessing chromosome chr10.
Finish processing chromosome chr10.
00:02:55.85
Finish saving chromosome chr10.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


...

Start multiprocessing chromosome chr8.
Finish processing chromosome chr8.
00:01:59.38
Finish saving chromosome chr8.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


8875it [00:00, 18374.72it/s]


Start multiprocessing chromosome chr9.
Finish processing chromosome chr9.
00:01:55.40
Finish saving chromosome chr9.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


9209it [00:00, 14081.05it/s]


Start multiprocessing chromosome chrX.
Finish processing chromosome chrX.
00:01:03.56
Finish saving chromosome chrX.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


# Exp 1027

In [11]:
dna_exp_key= '20230119'

# output dict for filename, file
output_df_dict = {}

exp_picked_spot_folder = [_f for _f in picked_spot_folders if dna_exp_key in _f][0]

if os.path.exists(exp_picked_spot_folder):
    file_list = [os.path.join(exp_picked_spot_folder, _fl) for _fl in os.listdir(exp_picked_spot_folder) if '.h5' in _fl]
    for file in file_list:
        output_df_dict[os.path.basename(file)]=pd.read_hdf(file)
        
# define output folder
output_folder = exp_picked_spot_folder.replace(para_subfolder, f'{para_subfolder}_sorted')
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    print ('Generating output folders')
    
# loop through all chr h5-df    
for fname, chr_df_all in output_df_dict.items():
    
    output_fname = os.path.join(output_folder, fname)
    chosen_chrom = chr_df_all['chr'].values[0]
    
    chr_df_cell_list = []
    # group by celltype, finalcellD
    for i, (name, group) in tqdm.tqdm(enumerate(chr_df_all.groupby(by=['celltype','finalcellID']))):
        chr_df_cell = copy.deepcopy(group)
        chr_df_cell_list.append(chr_df_cell)
        
    del chr_df_all
    print(f'Start multiprocessing chromosome {chosen_chrom}.')
    start_time = time.time()
    with mp.Pool(num_threads) as sort_pool:
        _res = sort_pool.starmap(aligner.sort_spots_by_fiber_density, zip(chr_df_cell_list), chunksize=1)
        sort_pool.close()
        sort_pool.join()
        sort_pool.terminate()

    elapsed_time = time.time() - start_time
    print(f'Finish processing chromosome {chosen_chrom}.')
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    sorted_chr_df_all = pd.concat(_res)
    sorted_chr_df_all.to_hdf(output_fname, key='df', mode='w')
    #del(sorted_chr_df_all)
    print(f'Finish saving chromosome {chosen_chrom}.')
    print(f'+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    


Generating output folders


6463it [00:00, 11438.94it/s]


Start multiprocessing chromosome chr1.
Finish processing chromosome chr1.
00:01:32.59
Finish saving chromosome chr1.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


7418it [00:00, 18498.79it/s]


Start multiprocessing chromosome chr10.
Finish processing chromosome chr10.
00:01:07.44
Finish saving chromosome chr10.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


...

Start multiprocessing chromosome chr8.
Finish processing chromosome chr8.
00:00:50.38
Finish saving chromosome chr8.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


6909it [00:00, 12607.67it/s]


Start multiprocessing chromosome chr9.
Finish processing chromosome chr9.
00:00:47.77
Finish saving chromosome chr9.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


6528it [00:00, 13295.36it/s]


Start multiprocessing chromosome chrX.
Finish processing chromosome chrX.
00:00:23.45
Finish saving chromosome chrX.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


# Exp 1103

In [12]:
dna_exp_key= '20230316'

# output dict for filename, file
output_df_dict = {}

exp_picked_spot_folder = [_f for _f in picked_spot_folders if dna_exp_key in _f][0]

if os.path.exists(exp_picked_spot_folder):
    file_list = [os.path.join(exp_picked_spot_folder, _fl) for _fl in os.listdir(exp_picked_spot_folder) if '.h5' in _fl]
    for file in file_list:
        output_df_dict[os.path.basename(file)]=pd.read_hdf(file)
        
# define output folder
output_folder = exp_picked_spot_folder.replace(para_subfolder, f'{para_subfolder}_sorted')
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    print ('Generating output folders')
    
# loop through all chr h5-df    
for fname, chr_df_all in output_df_dict.items():
    
    output_fname = os.path.join(output_folder, fname)
    chosen_chrom = chr_df_all['chr'].values[0]
    
    chr_df_cell_list = []
    # group by celltype, finalcellD
    for i, (name, group) in tqdm.tqdm(enumerate(chr_df_all.groupby(by=['celltype','finalcellID']))):
        chr_df_cell = copy.deepcopy(group)
        chr_df_cell_list.append(chr_df_cell)
        
    del chr_df_all
    print(f'Start multiprocessing chromosome {chosen_chrom}.')
    start_time = time.time()
    with mp.Pool(num_threads) as sort_pool:
        _res = sort_pool.starmap(aligner.sort_spots_by_fiber_density, zip(chr_df_cell_list), chunksize=1)
        sort_pool.close()
        sort_pool.join()
        sort_pool.terminate()

    elapsed_time = time.time() - start_time
    print(f'Finish processing chromosome {chosen_chrom}.')
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    sorted_chr_df_all = pd.concat(_res)
    sorted_chr_df_all.to_hdf(output_fname, key='df', mode='w')
    #del(sorted_chr_df_all)
    print(f'Finish saving chromosome {chosen_chrom}.')
    print(f'+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    


Generating output folders


7621it [00:00, 17974.11it/s]


Start multiprocessing chromosome chr1.
Finish processing chromosome chr1.
00:03:04.26
Finish saving chromosome chr1.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


7895it [00:00, 11854.38it/s]


Start multiprocessing chromosome chr10.
Finish processing chromosome chr10.
00:01:52.64
Finish saving chromosome chr10.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


...

Start multiprocessing chromosome chr8.
Finish processing chromosome chr8.
00:01:22.89
Finish saving chromosome chr8.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


7293it [00:00, 12839.78it/s]


Start multiprocessing chromosome chr9.
Finish processing chromosome chr9.
00:01:15.84
Finish saving chromosome chr9.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


7482it [00:00, 22267.81it/s]


Start multiprocessing chromosome chrX.
Finish processing chromosome chrX.
00:00:41.30
Finish saving chromosome chrX.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
